# Django Day-4 도서관 서비스 구현

## 세션 1: Django 프로젝트 설정 및 모델 생성

- **핵심 내용:**
    - Django 프로젝트를 설정하고, `Book` 모델을 생성하여 데이터베이스에 반영하는 과정을 학습했습니다.
    - 슈퍼유저를 생성하고, 관리자 페이지를 통해 도서 데이터를 관리하는 방법을 배웠습니다.
    - **주요 개념:** Django의 프로젝트 및 앱 구조, 기본적인 모델 정의와 마이그레이션
- **중요 포인트:**
    - Django 프로젝트 구조를 이해하고, `models.py`에서 모델을 정의한 후, 이를 데이터베이스에 적용하는 마이그레이션 과정이 중요합니다.

**Step 1: 가상환경 설정**
```py
# python -m venv venv #
# source venv/bin/activate  
# Windows: cmd : .\myenv\Scripts\activate
# powershell : .\myenv\Scripts\activate.bat
pip install django
```

**Step 2: 프로젝트 시작**
```py
django-admin startproject bookproject
cd bookproject
python manage.py startapp books
```
* bookproject 디렉토리 구조
```md
bookproject/
├── manage.py
├── bookproject/
│   ├── __init__.py
│   ├── settings.py
│   ├── urls.py
│   └── wsgi.py
└── books/
    ├── __init__.py
    ├── admin.py
    ├── apps.py
    ├── models.py
    ├── tests.py
    └── views.py
```

**Step 3: book 모델 정의**
* books/models.py
```py
from django.db import models

class Book(models.Model):
    title = models.CharField(max_length=200, verbose_name='제목')
    author = models.CharField(max_length=100, verbose_name='저자')
    publication_date = models.DateField(verbose_name='출판일')

    def __str__(self):
        return self.title

    class Meta:
        verbose_name = '도서'
        verbose_name_plural = '도서들'
```

**Step 4: settings.py 성정**
```py
INSTALLED_APPS에 'books' 앱 추가:
TEMPLATES > "DIRS": [BASE_DIR / "templates"], # 템플릿 디렉터리 추가
LANGUAGE_CODE = "ko-kr", TIME_ZONE = "Asia/Seoul"
STATIC_URL > STATICFILES_DIRS = [BASE_DIR / 'static']  # static 디렉터리 추가
```

**Step 5:makemigrations 추가**
```py
* python manage.py makemigrations
* python manage.py migrate
* ?: (staticfiles.W004) The directory 'D:\python\Oreum-Camp_Python\Django\CBV\bookproject\static' in the STATICFILES_DIRS setting does not exist. 오류 발생
* mkdir static > static 폴더추가
```

**Step 6:슈퍼유저 생성**
* python manage.py createsuperuser

**Step 7:서버 실행 테스트**
* python manage.py runserver

**Step 8:도서 추가 테스트**
* http://127.0.0.1:8000/admin/ 접속해서, 도서들 항목의 추가를 진행



## 세션 2: ListView를 활용한 도서 목록 구현

- **핵심 내용:**
    - Django의 `ListView`를 사용하여 도서 목록을 표시하는 방법을 학습했습니다.
    - FBV와 CBV의 차이점을 비교하면서, CBV의 간결함과 재사용성을 경험했습니다.
    - **주요 개념:** `ListView`, `get_queryset()`, `context_object_name`, 템플릿에서의 리스트 출력
- **중요 포인트:**
    - `ListView`는 데이터 조회와 템플릿 렌더링을 자동으로 처리하며, `get_queryset()`을 통해 필터링이나 정렬을 쉽게 커스터마이징할 수 있습니다.

**Step 1: ordering 사용** 샘플 코드 참고
* books/admin.py 파일을 열고 ordering 옵션을 추가합니다.
```py
from django.contrib import admin
from .models import Book

@admin.register(Book)
class BookAdmin(admin.ModelAdmin):
    list_display = ('title', 'author', 'publication_date')
    search_fields = ('title', 'author')
    list_filter = ('publication_date',)
    ordering = ('-publication_date',)  # 출판일을 기준으로 내림차순 정렬

```

**Step 2: ListViews 연동**
* books/views.py 
```py
from django.views.generic import ListView
from .models import Book

class BookListView(ListView):
    model = Book
    template_name = 'books/book_list.html'
    context_object_name = 'books'
    ordering = ['-publication_date']  # 출판일 기준 내림차순 정렬
```

**Step 3: urls 연동**
* books/urls.py 
```py
from django.urls import path
from .views import BookListView

app_name = 'books'

urlpatterns = [
    path('', BookListView.as_view(), name='book_list'),
]
```
* bookproject/urls.py
```py
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('books/', include('books.urls')),
]

```
**Step 4: 간단한 목록 템플릿 작성**
* mkdir -p templates/books
* templates/base.html
```py
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block title %}도서 관리 시스템{% endblock %}</title>
</head>
<body>
    <header>
        <h1>도서 관리 시스템</h1>
        <nav>
            <ul>
                <li><a href="{% url 'main' %}">홈</a></li>
                <li><a href="{% url 'books:book_list' %}">도서 목록</a></li>
            </ul>
        </nav>
    </header>
    <main>
        {% block content %}
        <!-- 여기에 개별 페이지의 내용이 추가됩니다 -->
        {% endblock %}
    </main>
    <footer>
        <p>&copy; 2024 도서 관리 시스템</p>
    </footer>
</body>
</html>
```
* templates/main.html
```py
{% extends 'base.html' %}

{% block title %}홈{% endblock %}

{% block content %}
    <h2>홈페이지</h2>
    <p>도서 관리 시스템에 오신 것을 환영합니다.</p>
    <p>상단 메뉴를 통해 원하는 페이지로 이동할 수 있습니다.</p>
{% endblock %}
```
* templates/books/book_list.html
```py
{% extends 'base.html' %}

{% block title %}도서 목록{% endblock %}

{% block content %}
    <h2>도서 목록</h2>
    <ul>
        {% for book in books %}
            <li>
                <strong>{{ book.title }}</strong> - {{ book.author }} ({{ book.publication_date }})
            </li>
        {% endfor %}
    </ul>
{% endblock %}
```

**Step 5: URL 패턴 설정 및 CBV 수정**
* bookproject/urls.py
```py
from django.contrib import admin
from django.urls import path, include
from books.views import MainView  # MainView를 임포트

urlpatterns = [
    path('admin/', admin.site.urls),
    path('books/', include('books.urls')),  # books 앱의 URL 패턴 포함
    path('', MainView.as_view(), name='main'),  # 메인 페이지 URL 패턴
]
```

**Step 6: views.py 작성**
* book/views.py
```py
from django.views.generic import TemplateView

class MainView(TemplateView):
    template_name = 'main.html'  # 메인 페이지로 사용할 템플릿 파일 지정
```

**Step 7: 서버 열어서 확인**
* python manage.py runserver

## 세션 3: DetailView를 활용한 도서 상세 페이지 구현

- **핵심 내용:**
    - Django의 `DetailView`를 사용하여 개별 도서의 상세 정보를 표시하는 방법을 학습했습니다.
    - FBV와 비교하여, CBV의 `DetailView`가 객체 조회와 예외 처리, 템플릿 렌더링을 간편하게 처리할 수 있음을 배웠습니다.
    - **주요 개념:** `DetailView`, `get_object()`, 템플릿에서 객체의 필드 출력
    - `DetailView`는 URL에서 전달된 고유 식별자(일반적으로 pk나 slug)를 기반으로 객체를 조회하며, 조회된 객체를 지정된 템플릿에 전달합니다.
- **중요 포인트:**
    - `DetailView`는 기본적으로 객체 조회를 자동으로 처리하며, 이를 통해 사용자가 특정 객체의 상세 정보를 쉽게 확인할 수 있습니다.

**Step 1: BookDetailView 작성**
* books/views.py
```py
from django.views.generic import DetailView
from .models import Book

class BookDetailView(DetailView):
    model = Book
    template_name = 'books/book_detail.html'
    context_object_name = 'book'
```

**Step 2: URL 패턴 설정**
* books/urls.py
```py
from django.urls import path
from .views import BookListView, BookDetailView

app_name = 'books'

urlpatterns = [
    path('', BookListView.as_view(), name='book_list'),
    path('<int:pk>/', BookDetailView.as_view(), name='book_detail'),
]
```

**Step 3: 간단한 상세 페이지 템플릿 작성**
* templates/books/book_detail.html
```py
{% extends 'base.html' %}

{% block title %}{{ book.title }} - 상세 정보{% endblock %}

{% block content %}
    <h1>{{ book.title }}</h1>
    <p><strong>저자:</strong> {{ book.author }}</p>
    <p><strong>출판일:</strong> {{ book.publication_date }}</p>
    <a href="{% url 'books:book_list' %}">목록으로 돌아가기</a>
{% endblock %}
```

**Step 4: 목록 페이지에 상세 페이지 링크 추가**
* templates/books/book_list.html
```py
{% extends 'base.html' %}

{% block title %}도서 목록{% endblock %}

{% block content %}
    <h2>도서 목록</h2>
    <ul>
        {% for book in books %}
            <li>
                <strong><a href="{% url 'books:book_detail' book.pk %}">{{ book.title }}</a></strong> - {{ book.author }} ({{ book.publication_date }})
            </li>
        {% endfor %}
    </ul>
{% endblock %}
```

**Step 5: 서버 실행 및 도서 상세 페이지 확인**
* python manage.py runserver

## 세션 4: CreateView를 활용한 도서 생성 기능 구현

- **핵심 내용:**
    - Django의 `CreateView`를 사용하여 새로운 도서를 추가하는 기능을 구현했습니다.
    - `ModelForm`과 `CreateView`를 활용해 폼 처리, 유효성 검사, 객체 저장을 자동화했습니다.
    - **주요 개념:** `CreateView`, `ModelForm`, `get_success_url()`, 폼 처리와 유효성 검사
- **중요 포인트:**
    - `CreateView`는 폼 처리와 객체 저장을 자동으로 처리하므로, 코드가 간결해지며, `get_success_url()`을 통해 리다이렉션 경로를 쉽게 설정할 수 있습니다.

**Step 1: BookForm 작성**
* books/forms.py -> import forms, .models import Book -> 
```py
class BookForm(forms.ModelForm):
    class Meta:
        model = Book
        fields = ['title', 'author', 'publication_date']
```
**Step 2: BookCreateView 작성**
* books/views.py -> import CreateView, import reverse_lazy,  import BookForm
* class BookListView(ListView) -> paginate_by = 1  # 한 페이지에 1개의 항목을 표시 (추가)
* class BookCreateView(CreateView) 작성
```py
    class BookCreateView(CreateView):   
    model = Book
    form_class = BookForm
    template_name = 'books/book_form.html'
    success_url = reverse_lazy('books:book_list')
```
**Step 3: URL 패턴 설정**
* books/urls.py
```py
path('new/', BookCreateView.as_view(), name='book_create'),
```
**Step 4: 간단한 도서 생성 템플릿 작성**
* templates/books/book_form.html
```py
{% extends 'base.html' %}

{% block title %}도서 추가{% endblock %}

{% block content %}
    <h2>도서 추가</h2>
    <form method="post">
        {% csrf_token %}
        {{ form.as_p }}
        <button type="submit">저장</button>
    </form>
    <a href="{% url 'books:book_list' %}">도서 목록으로 돌아가기</a>
{% endblock %}
```
**Step 5: 도서 목록 페이지에 도서 추가 링크 추가**
* templates/books/book_list.html
```py
{% extends 'base.html' %}

{% block title %}도서 목록{% endblock %}

{% block content %}
    <h2>도서 목록</h2>
    <a href="{% url 'books:book_create' %}">새 도서 추가</a>
    <ul>
        {% for book in books %}
            <li>
                <strong><a href="{% url 'books:book_detail' book.pk %}">{{ book.title }}</a></strong> - {{ book.author }} ({{ book.publication_date|date:"Y-m-d" }})
            </li>
        {% endfor %}
    </ul>

    <!-- 페이지네이션 링크 추가 -->
    <div class="pagination">
        <span class="step-links">
            {% if page_obj.has_previous %}
                <a href="?page=1">&laquo; 처음</a>
                <a href="?page={{ page_obj.previous_page_number }}">이전</a>
            {% endif %}

            <span class="current">
                페이지 {{ page_obj.number }} / {{ page_obj.paginator.num_pages }}
            </span>

            {% if page_obj.has_next %}
                <a href="?page={{ page_obj.next_page_number }}">다음</a>
                <a href="?page={{ page_obj.paginator.num_pages }}">마지막 &raquo;</a>
            {% endif %}
        </span>
    </div>
{% endblock %}
```
**Step 6: 출판일 기능 추가**
* books/forms.py
```py
from django import forms
from .models import Book

class BookForm(forms.ModelForm):
    class Meta:
        model = Book
        fields = ['title', 'author', 'publication_date']
        widgets = {
            'publication_date': forms.DateInput(attrs={'type': 'date'}),  # 날짜 선택기를 위한 위젯 설정
        }
        labels = {
            'title': '제목',
            'author': '저자',
            'publication_date': '출판일',
        }
```
**Step 7: 결과 확인**
* python manage.py runserver




## 세션 5: UpdateView를 활용한 도서 수정 기능 구현

- **핵심 내용:**
    - Django의 `UpdateView`를 사용하여 기존 도서 정보를 수정하는 기능을 구현했습니다.
    - `UpdateView`와 `CreateView`의 유사점과 차이점을 학습하며, 객체 수정 작업을 간편하게 처리하는 방법을 배웠습니다.
    - **주요 개념:** `UpdateView`, 폼 재사용, `get_success_url()`을 통한 수정 후 리다이렉션
- **중요 포인트:**
    - `UpdateView`를 통해 기존 객체의 수정 작업을 쉽게 처리할 수 있으며, `CreateView`와 템플릿을 재사용할 수 있다는 점에서 코드의 효율성을 높일 수 있습니다.

**Step 1: BookUpdateView 작성**
* books/views.py 
```py
from django.views.generic.edit import UpdateView
from django.urls import reverse_lazy
from .models import Book
from .forms import BookForm

class BookUpdateView(UpdateView):
    model = Book
    form_class = BookForm
    template_name = 'books/book_form.html'

    def get_success_url(self):
        return reverse_lazy('books:book_detail', kwargs={'pk': self.object.pk})

```
**Step 2: URL 패턴 설정***
* books/urls.py
```py
from django.urls import path
from .views import BookListView, BookDetailView, BookCreateView, BookUpdateView

app_name = 'books'

urlpatterns = [
    path('', BookListView.as_view(), name='book_list'),
    path('<int:pk>/', BookDetailView.as_view(), name='book_detail'),
    path('new/', BookCreateView.as_view(), name='book_create'),
    path('<int:pk>/edit/', BookUpdateView.as_view(), name='book_update'),
]

```
**Step 3: book_form.html 템플릿 수정**
* books/templates/books/book_form.html
```py
{% extends 'base.html' %}

{% block title %}
    {% if view.object.pk %}도서 정보 수정{% else %}도서 추가{% endif %}
{% endblock %}

{% block content %}
    <h2>
        {% if view.object.pk %}도서 정보 수정{% else %}도서 추가{% endif %}
    </h2>
    <form method="post">
        {% csrf_token %}
        {{ form.as_p }}
        <button type="submit">
            {% if view.object.pk %}수정{% else %}추가{% endif %}
        </button>
    </form>
    <a href="{% url 'books:book_list' %}">도서 목록으로 돌아가기</a>
{% endblock %}

```
**Step 4: book_detail.html에 수정 링크 추가**
* templates/books/book_detail.html
```py
{% extends 'base.html' %}

{% block title %}{{ book.title }} - 상세 정보{% endblock %}

{% block content %}
    <h2>{{ book.title }}</h2>
    <p><strong>저자:</strong> {{ book.author }}</p>
    <p><strong>출판일:</strong> {{ book.publication_date|date:"Y-m-d" }}</p>
    <a href="{% url 'books:book_update' book.pk %}">도서 정보 수정</a>
    <br>
    <a href="{% url 'books:book_list' %}">도서 목록으로 돌아가기</a>
{% endblock %}
```
**Step 5: 서버 실행 및 도서 수정 기능 테스트**
* python manage.py runserver

## 세션 6: DeleteView를 활용한 도서 삭제 기능 구현

- **핵심 내용:**
    - Django의 `DeleteView`를 사용하여 도서를 삭제하는 기능을 구현했습니다.
    - 삭제 전 확인 페이지를 제공하여 사용자가 실수로 데이터를 삭제하지 않도록 하는 방법을 배웠습니다.
    - **주요 개념:** `DeleteView`, 삭제 확인 템플릿, 삭제 후 리다이렉션
- **중요 포인트:**
    - `DeleteView`는 객체 삭제와 리다이렉션을 자동으로 처리하며, 사용자가 삭제를 신중히 결정할 수 있도록 삭제 확인 페이지를 제공하는 것이 중요합니다.

**Step 1: BookDeleteView 작성**
* books/views.py
```py
from django.views.generic import DeleteView
from django.urls import reverse_lazy
from .models import Book

class BookDeleteView(DeleteView):
    model = Book
    template_name = 'books/book_confirm_delete.html'
    success_url = reverse_lazy('books:book_list')
```
**Step 2: URL 패턴 설정**
* books/urls.py
```py
from django.urls import path
from .views import BookListView, BookDetailView, BookCreateView, BookUpdateView, BookDeleteView

app_name = 'books'

urlpatterns = [
    path('', BookListView.as_view(), name='book_list'),
    path('<int:pk>/', BookDetailView.as_view(), name='book_detail'),
    path('new/', BookCreateView.as_view(), name='book_create'),
    path('<int:pk>/edit/', BookUpdateView.as_view(), name='book_update'),
    path('<int:pk>/delete/', BookDeleteView.as_view(), name='book_delete'),
]
```
**Step 3: 삭제 확인 템플릿 작성**
* templates/books/book_confirm_delete.html
```py
{% extends 'base.html' %}

{% block title %}도서 삭제{% endblock %}

{% block content %}
    <h2>도서 삭제</h2>
    <p>정말로 이 도서를 삭제하시겠습니까?</p>
    <p><strong>{{ book.title }}</strong> - {{ book.author }}</p>
    <form method="post">
        {% csrf_token %}
        <button type="submit">삭제</button>
        <a href="{% url 'books:book_detail' book.pk %}">취소</a>
    </form>
{% endblock %}
```
**Step 4: 상세 페이지에 삭제 링크 추가**
* templates/books/book_detail.html
```py
{% extends 'base.html' %}

{% block title %}{{ book.title }} - 상세 정보{% endblock %}

{% block content %}
    <h2>{{ book.title }}</h2>
    <p><strong>저자:</strong> {{ book.author }}</p>
    <p><strong>출판일:</strong> {{ book.publication_date|date:"Y-m-d" }}</p>
    <a href="{% url 'books:book_update' book.pk %}">도서 정보 수정</a>
    <br>
    <a href="{% url 'books:book_delete' book.pk %}">도서 삭제</a> <!-- 삭제 링크 추가 -->
    <br>
    <a href="{% url 'books:book_list' %}">도서 목록으로 돌아가기</a>
{% endblock %}
```
**Step 5: 서버 실행 및 도서 삭제 기능 테스트**
* python manage.py runserver


### **추가 실습 과제**

CRUD 기능을 모두 학습한 후, 다음의 실습 과제를 통해 Django CBV와 웹 애플리케이션 개발에 대한 이해를 더욱 깊이 있게 다질 수 있습니다.

1. **검색 및 필터링 기능 추가:**
    - `ListView`에서 사용자가 검색어를 입력하면, 제목이나 저자에 해당하는 도서만 목록에 표시되도록 검색 기능을 구현하세요.
    - 추가적으로, 출판일이나 저자별로 도서를 필터링할 수 있는 기능을 구현해보세요.
2. **페이지네이션 구현:**
    - `ListView`에 페이지네이션을 추가하여, 한 페이지에 표시할 도서 수를 제한하고, 페이지 이동 버튼을 추가해보세요.
    - 페이지네이션이 잘 작동하는지 테스트하고, 다양한 시나리오에서의 동작을 확인하세요.
3. **사용자 인증 및 접근 제어:**
    - 로그인 기능을 추가하여, 로그인한 사용자만 도서를 추가, 수정, 삭제할 수 있도록 접근 제어를 설정해보세요.
    - `LoginRequiredMixin`을 사용하여 비로그인 사용자가 접근할 수 없도록 설정하고, 로그인 페이지로 리다이렉트되도록 설정하세요.
4. **사용자 등록 및 관리:**
    - 사용자 등록 기능을 추가하여, 새로운 사용자가 회원가입할 수 있도록 하세요.
    - 관리자 페이지를 통해 사용자 정보를 관리할 수 있는 기능을 확인하고, 필요 시 커스터마이징하세요.
5. **부트스트랩을 활용한 템플릿 스타일링:**
    - 부트스트랩을 적용하여, 현재의 기본 HTML 템플릿을 더 아름답고 사용자 친화적으로 꾸며보세요.
    - 버튼, 네비게이션 바, 폼 등의 UI 요소에 부트스트랩 클래스를 적용하여 일관된 디자인을 유지하세요.
6. **에러 페이지 및 사용자 피드백 개선:**
    - 404, 500 등의 에러 페이지를 커스터마이징하여 사용자에게 더 나은 피드백을 제공하세요.
    - 도서 생성, 수정, 삭제 등의 작업이 완료되었을 때, 사용자에게 성공 메시지를 보여주는 기능을 추가해보세요.

---

**마무리:**

- 이 추가 실습 과제들을 통해 Django CBV와 웹 개발에 대한 이해도를 더욱 높일 수 있습니다. 실습을 통해 다양한 상황에 맞는 기능을 구현하고, 실무에서 필요한 여러 가지 개념들을 직접 체험해보세요.
- 이 과정이 완료되면, Django Rest Framework(DRF)나 배포와 같은 더 고급 주제로 넘어가 웹 애플리케이션을 완성도 있게 만들어볼 수 있습니다.

## 이미지 업로드
### 목표:

* 도서 정보에 이미지를 업로드하고 이를 표시하는 기능을 구현합니다.
* Django 프로젝트에서 이미지 파일을 처리하고 표시하는 방법을 익힙니다.

### 준비 사항:

* `pip install Pillow`를 통해 이미지 처리 라이브러리인 Pillow를 설치합니다.

### 실습 단계:

**step 1: 모델 수정**
* books/models.py
```py

from django.db import models

class Book(models.Model):
    title = models.CharField(max_length=200)
    author = models.CharField(max_length=200)
    publication_date = models.DateField()
    genre = models.CharField(max_length=100)
    summary = models.TextField()
    image = models.ImageField(upload_to='book_images/', blank=True, null=True)  # 이미지 필드 추가

    def __str__(self):
        return self.title

```

**step 2: 마이그레이션 실행**
```shell
python manage.py makemigrations
python manage.py migrate
```


**step 3: 미디어 파일 설정**
* bookproject/media/book_images/ 폴더 생성
* setting.py 수정
```py
import os

MEDIA_URL = '/media/'
MEDIA_ROOT = os.path.join(BASE_DIR, 'media')
```

**step 4: URL 설정**
* bookproject/urls.py
```py
# urls.py

from django.conf import settings
from django.conf.urls.static import static

urlpatterns = [
    # 여기에 기존의 URL 패턴들이 들어갑니다.
]

if settings.DEBUG:
    urlpatterns += static(settings.MEDIA_URL, document_root=settings.MEDIA_ROOT)

```

**step 5: 폼 수정**
* books/forms.py
```py

from django import forms
from .models import Book

class BookForm(forms.ModelForm):
    class Meta:
        model = Book
        fields = ['title', 'author', 'publication_date', 'genre', 'summary', 'image']  # 이미지 필드 추가

```
**step 6: 템플릿 수정**
* books/templates/books
* book_form.html 파일 수정
```py
<form method="post" enctype="multipart/form-data">
    {% csrf_token %}
    {{ form.as_p }}
    <button type="submit">저장</button>
</form>
```
* book_detail.html 파일 수정: 요약 하단에 작성
```py

{% if book.image %}
    <img src="{{ book.image.url }}" alt="{{ book.title }} 이미지">
{% else %}
    <p>이미지가 없습니다.</p>
{% endif %}

```
**step 7: 확인**
```shell
python manage.py runserver
```


